In [ ]:
import pandas as pd
import numpy as np
import os

# See github.com/hora-search/horapy
try:
    import horapy
except:
    os.system('pip install horapy')
    
from horapy import HNSWIndex

from numpy.random import Generator, PCG64
rng = Generator(PCG64())

# Fast Approximate NN performance comparison

A first cut of the retrieval times for one current FANN implementation compared to brute force linear search in a dataset of embedding vectors.  The example uses the IMDB data set as indexed by embeddings from the 'all-MiniLM-L6-v2' sentenceTransformer model---a set of 50,000 items of 384 features. 

tl;dr - Linear search (and sorting also) on this "small" set are so fast that it would be hard to beat the timings of brute force code.

JMA Feb 2024

In [ ]:
# Load the existing IMDB vectorized data
DATA_DIR = '/mnt/512G_hd/data/IMDB/'
text_df = pd.read_parquet(DATA_DIR + 'IMDB_sentiment.parquet') 
    
print(text_df.shape)
text_df.head()

In [ ]:
# Extract the vector field and expand it to multiple rows. 
n_samples, embedding_dim = text_df.shape
sample_ar = np.empty((n_samples, 384), 'float')
for i in range(n_samples):
    x = text_df.values[i,2]
    sample_ar[i] = x # [np.newaxis,:]    # Works but adding the dimension isn't necessary
    
sample_ar.shape

In [ ]:
# Select one item as a target, and build an index to find distances

n_samples, em_dimension = sample_ar.shape

embedding_index = HNSWIndex(em_dimension, "usize")

target = rng.choice(n_samples)

for i in range(n_samples):
    if i != target:
        # Add an embedding vector to the index
        a_sample = np.float32(sample_ar[i,:])
        # print(a_sample.shape, type(a_sample))
        embedding_index.add(a_sample,i)
    else:
        chosen = np.float32(sample_ar[i,:])
        
embedding_index.build('euclidean')


In [ ]:
%%time
y = embedding_index.search(chosen,49999)
# Lookup all samples in order of distance 
y[:10]

In [ ]:
%%time
distances = np.empty((n_samples,))

for i in range(n_samples):
    distances[i] = np.dot(chosen, sample_ar[i])
# Making a comparison with just running a linear traversal via dot products through the data


In [ ]:
%%time
d2 = distances.copy()
d2.sort()
d2[:-10]
# Comparison - to be fair the time to sort the distances should be counted. 

In [ ]:
# Range of dot product values
pd.DataFrame(distances).describe()